In [15]:
import decimal
import json
import logging
import os
import pprint
import time
import boto3
from boto3.dynamodb.conditions import Key, Attr
from botocore.exceptions import ClientError

logger = logging.getLogger(__name__)
dyn_resource = boto3.resource('dynamodb')

channel_table = dyn_resource.Table("channel-dev_cta_20221225-classu_ted")
post_table = dyn_resource.Table("post-dev_cta_20221225-classu_ted")
class_user_table = dyn_resource.Table("class_user-dev_cta_20221225-classu_ted")

In [16]:
def do_batch_get(batch_keys):
    """
    Gets a batch of items from Amazon DynamoDB. Batches can contain keys from
    more than one table.
    When Amazon DynamoDB cannot process all items in a batch, a set of unprocessed
    keys is returned. This function uses an exponential backoff algorithm to retry
    getting the unprocessed keys until all are retrieved or the specified
    number of tries is reached.
    :param batch_keys: The set of keys to retrieve. A batch can contain at most 100
                       keys. Otherwise, Amazon DynamoDB returns an error.
    :return: The dictionary of retrieved items grouped under their respective
             table names.
    """
    tries = 0
    max_tries = 5
    sleepy_time = 1  # Start with 1 second of sleep, then exponentially increase.
    retrieved = {key: [] for key in batch_keys}
    while tries < max_tries:
        response = dyn_resource.batch_get_item(RequestItems=batch_keys)
        # Collect any retrieved items and retry unprocessed keys.
        for key in response.get('Responses', []):
            retrieved[key] += response['Responses'][key]
        unprocessed = response['UnprocessedKeys']
        if len(unprocessed) > 0:
            batch_keys = unprocessed
            unprocessed_count = sum(
                [len(batch_key['Keys']) for batch_key in batch_keys.values()])
            logger.info(
                "%s unprocessed keys returned. Sleep, then retry.",
                unprocessed_count)
            tries += 1
            if tries < max_tries:
                logger.info("Sleeping for %s seconds.", sleepy_time)
                time.sleep(sleepy_time)
                sleepy_time = min(sleepy_time * 2, 32)
        else:
            break

    return retrieved

In [17]:
def get_batch_data(class_id, user_id_list):
    batch_keys = {
        class_user_table.name: {
            "Keys": [ { "partition_key": f"class@{class_id}", "sort_key": f"user#user@{i}" } for i in user_id_list ]
        }
    }
    
    try:
        retrieved = do_batch_get(batch_keys)
        for response_table, response_items in retrieved.items():
            logger.info("Got %s items from %s.", len(response_items), response_table)
    except ClientError:
#         logger.exception(
#             "Couldn't get items from %s and %s.", movie_table.name, actor_table.name)
        raise
    else:
        return retrieved

In [18]:
class_id = "1"
user_id_list = ["1","1234799","2","3","777","Ruben","chad"]
batch_data = get_batch_data(class_id, user_id_list)

In [19]:
print(batch_data)

{'class_user-dev_cta_20221225-classu_ted': [{'updated_at': Decimal('1673500667425'), 'profile_image': 'https://picsum.photos/200', 'user_id': 'Ruben', 'profile_border_color': '', 'grade': Decimal('1'), 'created_at': Decimal('1673500667425'), 'nickname': '1yfgvyhvrtyvyurt', 'partition_key': 'class@1', 'total_badge_count': Decimal('0'), 'sort_key': 'user#user@Ruben', 'class_id': '1'}, {'created_at': Decimal('1673449949505'), 'grade': Decimal('1'), 'partition_key': 'class@1', 'sort_key': 'user#user@1234799', 'class_id': '1', 'main_badge_url': 'https://classu-files.s3.ap-northeast-2.amazonaws.com/badge/badge_a.png', 'updated_at': Decimal('1673449949505'), 'profile_image': 'https://picsum.photos/200', 'profile_border_color': 'blue', 'user_id': '1234799', 'nickname': 'hi', 'total_badge_count': Decimal('0')}, {'updated_at': Decimal('1673498279907'), 'profile_image': 'https://picsum.photos/200', 'user_id': '3', 'profile_border_color': '', 'grade': Decimal('1'), 'created_at': Decimal('167349827

In [35]:
def batch_load(class_id, user_id_list):
    class_user_list = []
    
    if not user_id_list:
        return class_user_list
    
    chunked_list = [ user_id_list[i:i+100] for i in range(0, len(user_id_list), 100) ]
    
    for i in chunked_list:
        class_user_list += get_batch_data(class_id, i)[class_user_table.name]
        
    return class_user_list

In [38]:
class_id = "1"
user_id_list = ["1","1234799","2","3","777","Ruben","chad", "1"]
batch_data = batch_load(class_id, list(set(user_id_list)))

print(batch_data)

[{'updated_at': Decimal('1673500667425'), 'profile_image': 'https://picsum.photos/200', 'user_id': 'Ruben', 'profile_border_color': '', 'grade': Decimal('1'), 'created_at': Decimal('1673500667425'), 'nickname': '1yfgvyhvrtyvyurt', 'partition_key': 'class@1', 'total_badge_count': Decimal('0'), 'sort_key': 'user#user@Ruben', 'class_id': '1'}, {'created_at': Decimal('1673449949505'), 'grade': Decimal('1'), 'partition_key': 'class@1', 'sort_key': 'user#user@1234799', 'class_id': '1', 'main_badge_url': 'https://classu-files.s3.ap-northeast-2.amazonaws.com/badge/badge_a.png', 'updated_at': Decimal('1673449949505'), 'profile_image': 'https://picsum.photos/200', 'profile_border_color': 'blue', 'user_id': '1234799', 'nickname': 'hi', 'total_badge_count': Decimal('0')}, {'updated_at': Decimal('1673498279907'), 'profile_image': 'https://picsum.photos/200', 'user_id': '3', 'profile_border_color': '', 'grade': Decimal('1'), 'created_at': Decimal('1673498279907'), 'nickname': 'ruben', 'partition_key

In [52]:
batch_keys = {
        class_user_table.name: {
            "Keys": [ { "partition_key": f"class@1", "sort_key": f"user#user@1" }, { "partition_key": f"class@1", "sort_key": f"user#user@Ruben" } ]
        }
    }
result = dyn_resource.batch_get_item(RequestItems=batch_keys)
print(result)

{'Responses': {'class_user-dev_cta_20221225-classu_ted': [{'updated_at': Decimal('1673500667425'), 'profile_image': 'https://picsum.photos/200', 'user_id': 'Ruben', 'profile_border_color': '', 'grade': Decimal('1'), 'created_at': Decimal('1673500667425'), 'nickname': '1yfgvyhvrtyvyurt', 'partition_key': 'class@1', 'total_badge_count': Decimal('0'), 'sort_key': 'user#user@Ruben', 'class_id': '1'}, {'grade': Decimal('1'), 'created_at': Decimal('1674614135888'), 'partition_key': 'class@1', 'sort_key': 'user#user@1', 'class_id': '1', 'main_badge_url': 'https://classu-files.s3.ap-northeast-2.amazonaws.com/badge/badge_a.png', 'updated_at': Decimal('1674614135888'), 'profile_image': 'https://picsum.photos/200', 'user_id': '1', 'profile_border_color': 'symbol_purple', 'nickname': '1', 'total_badge_count': Decimal('0')}]}, 'UnprocessedKeys': {}, 'ResponseMetadata': {'RequestId': 'CDEREBNV0MCMDJAFUH9LOMSJ9JVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'd

In [46]:
def batch_load(class_id, user_id_list):
    class_user_list = []
    
    if not user_id_list:
        return class_user_list
    
    chunked_list = [ user_id_list[i:i+100] for i in range(0, len(user_id_list), 100) ]
    
    for i in chunked_list:
        class_user_list += batch_100_load(class_id, i)
        
    return class_user_list


def batch_100_load(class_id, user_id_list):
    batch_keys = {
        class_user_table.name: {
            "Keys": [ { "partition_key": f"class@{class_id}", "sort_key": f"user#user@{i}" } for i in user_id_list ]
        }
    }
    
    try:
        retrieved = dyn_resource.batch_get_item(RequestItems=batch_keys)
    except ClientError:
#         logger.exception(
#             "Couldn't get items from %s and %s.", movie_table.name, actor_table.name)
        raise
    else:
        return retrieved["Responses"][class_user_table.name]
    

class_id = "1"
user_id_list = ["1","1234799","2","3","777","Ruben","chad", "1"]
batch_data = batch_load(class_id, list(set(user_id_list)))

print(batch_data)

[{'updated_at': Decimal('1673509070462'), 'profile_image': 'https://picsum.photos/200', 'profile_border_color': '', 'user_id': '2', 'created_at': Decimal('1673509070462'), 'grade': Decimal('1'), 'nickname': '1', 'partition_key': 'class@1', 'total_badge_count': Decimal('0'), 'sort_key': 'user#user@2', 'class_id': '1'}, {'grade': Decimal('1'), 'created_at': Decimal('1674614135888'), 'partition_key': 'class@1', 'sort_key': 'user#user@1', 'class_id': '1', 'main_badge_url': 'https://classu-files.s3.ap-northeast-2.amazonaws.com/badge/badge_a.png', 'updated_at': Decimal('1674614135888'), 'profile_image': 'https://picsum.photos/200', 'user_id': '1', 'profile_border_color': 'symbol_purple', 'nickname': '1', 'total_badge_count': Decimal('0')}, {'updated_at': Decimal('1673839359248'), 'profile_image': 'https://picsum.photos/200', 'user_id': 'chad', 'profile_border_color': '', 'grade': Decimal('3'), 'created_at': Decimal('1673839359248'), 'nickname': 'chad', 'partition_key': 'class@1', 'total_badg

In [53]:
test_table = dyn_resource.Table("profile-myspace8")

def test_batch_load(user_id_list):
    item_list = []
    
    if not user_id_list:
        return item_list
    
    chunked_list = [ user_id_list[i:i+100] for i in range(0, len(user_id_list), 100) ]
    
    for i in chunked_list:
        item_list += test_batch_100_load(i)
        
    return item_list


def test_batch_100_load(user_id_list):
    batch_keys = {
        test_table.name: {
            "Keys": [ { "id": "1", "userId": i } for i in user_id_list ]
        }
    }
    
    try:
        retrieved = dyn_resource.batch_get_item(RequestItems=batch_keys)
    except ClientError:
#         logger.exception(
#             "Couldn't get items from %s and %s.", movie_table.name, actor_table.name)
        raise
    else:
        return retrieved["Responses"][test_table.name]
    
    
user_id_list = [ str(i) for i in range(1, 101) ]
batch_data = test_batch_load(user_id_list)

In [49]:
print(batch_data)

[{'content': '국민으로부터 나온다.\n\n조문체계도버튼연혁\n 제2조 ①대한민국의 국민이 되는 요건은 법률로 정한다.\n\n②국가는 법률이 정하는 바에 의하여 재외국민을 보호할 의무를 진다.\n\n조문체계도버튼연혁\n 제3조 대한민국의 영토는 한반도와 그 부속도서로 한다.\n\n조문체계도버튼연혁\n 제4조 대한민국은 통일을 지향하며, 자유민주적 기본질서에 입각한 평화적 통일정책을 수립하고 이를 추진한다.\n\n조문체계도버튼연혁\n 제5조 ①대한민국은 국제평화의 유지에 노력하고 침략적 전쟁을 부인한다.\n\n②국군은 국가의 안전보장과 국토방위의 신성한 의무를 수행함을 사명으로 하며, 그 정치적 중립성은 준수된다.\n\n조문체계도버튼연혁\n 제6조 ①헌법에 의하여 체결ㆍ공포된 조약과 일반적으로 승인된 국제법규는 국내법과 같은 효력을 가진다.\n\n②외국인은 국제법과 조약이 정하는 바에 의하여 그 지위가 보장된다.\n\n조문체계도버튼연혁\n 제7조 ①공무원은 국민전체에 대한 봉사자이며, 국민에 대하여 책임을 진다.\n\n②공무원의 신분과 정치적 중립성은 법률이 정하는 바에 의하', 'id': '1', 'userId': '59', 'title': '며드는 찬공기들 기다릴께 언제라도 출발할 수 있도록 항상 엔진을 켜둘께 너와 만난 시간보다 많은 시간이 흐르고'}, {'content': '도버튼연혁\n 제6조 ①헌법에 의하여 체결ㆍ공포된 조약과 일반적으로 승인된 국제법규는 국내법과 같은 효력을 가진다.\n\n②외국인은 국제법과 조약이 정하는 바에 의하여 그 지위가 보장된다.\n\n조문체계도버튼연혁\n 제7조 ①공무원은 국민전체에 대한 봉사자이며, 국민에 대하여 책임을 진다.\n\n②공무원의 신분과 정치적 중립성은 법률이 정하는 바에 의하여 보장된다.\n\n조문체계도버튼연혁생활법령버튼\n 제8조 ①정당의 설립은 자유이며, 복수정당제는 보장된다.\n\n②정당은 그 목적ㆍ조직과 활동이 민주적이어야 하며, 국민의 정치적 의사형성에 참여하는데 필요한 조직을 가져야 한다